# Quantum Max Flow

Having experimented with the max flow algorithm in Notebook 1 using Google's `ortools` library, we will now attempt to apply this algorithm to the quantum world.

For this purpose, we will use DWave.

In [1]:
# Load Data
import geopandas as gpd

gdf_edges = gpd.read_file("../data/gdf_edges_bi.geojson") # Read bi-directional edges
gdf_nodes = gpd.read_file("../data/gdf_nodes.geojson") # Read nodes

In [2]:
from dimod import ExactSolver, BinaryQuadraticModel

# Create a simple BQM (Binary Quadratic Model)
bqm = BinaryQuadraticModel({'x': -1, 'y': -1}, {('x', 'y'): 2}, 0.0, 'BINARY')

# Use local classical solver
sampler = ExactSolver()
sampleset = sampler.sample(bqm)

print(sampleset)

   x  y energy num_oc.
1  1  0   -1.0       1
3  0  1   -1.0       1
0  0  0    0.0       1
2  1  1    0.0       1
['BINARY', 4 rows, 4 samples, 2 variables]


In [4]:
from dwave.samplers import SimulatedAnnealingSampler

# Define graph edges and weights (undirected)
edges = {
    ('A', 'B'): 1,
    ('A', 'C'): 1,
    ('B', 'D'): 1,
    ('C', 'D'): 1,
    ('B', 'C'): 2
}

# Fix node 'A' to be on the source side (value 0)
# Fix node 'D' to be on the sink side (value 1)
fixed = {'A': 0, 'D': 1}
penalty = 10  # large enough to enforce constraints

# Build the QUBO
bqm = BinaryQuadraticModel('BINARY')

for (u, v), w in edges.items():
    bqm.add_interaction(u, v, 2 * w)
    bqm.add_variable(u, -w)
    bqm.add_variable(v, -w)

# Add hard constraints
for var, value in fixed.items():
    if value == 0:
        bqm.add_variable(var, penalty)
    else:
        bqm.add_variable(var, -penalty)

# Solve using simulated annealing (local, no token needed)
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm, num_reads=100)
best = sampleset.first.sample

# Output
print("Min-Cut Partition:")
for node, val in best.items():
    side = "Source side" if val == 0 else "Sink side"
    print(f"  {node}: {side}")

# Show cut edges
cut_edges = [(u, v) for (u, v), _ in edges.items() if best[u] != best[v]]
print("\nCut Edges:")
for u, v in cut_edges:
    print(f"  {u} — {v}")


Min-Cut Partition:
  A: Source side
  B: Source side
  C: Sink side
  D: Sink side

Cut Edges:
  A — C
  B — D
  B — C
